# Noise Defense (Table 6)

In [ ]:
import numpy as np
import os
import pandas as pd

In [ ]:
dataset_name = 'ADULT'
experiments = {
    46: 'TabLeak',
    0: 'Inverting Gradients'
}
batch_size = 32
noise_scales = [1e-3, 1e-2, 1e-1]
noise_type = 'Gaussian'
random_seed = 42

In [ ]:
table_dataframe = pd.DataFrame()
table_dataframe['Noise Scale'] = noise_scales
for experiment_number, experiment_name in experiments.items():
    post_selection = 1 if experiment_number == 0 else 30
    base_path = f'experiment_data/DP_experiments/{dataset_name}/experiment_{experiment_number}/DP_data_all_{experiment_number}_{dataset_name}_50_{post_selection}_1500_128_{noise_scales[-1]}_0.319_{random_seed}.npy'
    if os.path.isfile(base_path):
        experiment_data = np.load(base_path)[0, :, :, 0]
    else:
        print(f'Experiment {experiment_name} not found')
        continue
    
    data_to_add = [(np.around(100 - 100*mean, 1), np.around(100*std, 1)) for mean, std in experiment_data[5, :, :2]]
    table_dataframe[experiment_name] = data_to_add

# load the trained network accuracies
load_path = f'experiment_data/DP_experiments/ADULT/trained_model_accuracies_ADULT_{random_seed}.npy'
if os.path.isfile(load_path):
    mean_accuracies, std_accuracies = np.around(100*np.mean(np.load(load_path)[1:, :, -1, 0], axis=1), 1), np.around(100*np.std(np.load(load_path)[1:, :, -1, 0], axis=1), 1)
    data_to_add = [(m, s) for m, s in zip(mean_accuracies, std_accuracies)]
    table_dataframe['Network Accuracy'] = data_to_add
else:
    print('Trained network data missing, please run the python script train_noise_defended_networks.py')

In [ ]:
table_dataframe